<table>
  <tr>
    <td><div align="left"><font size="30" >Practical 06 - Robotic Vision</font></div></td>
  </tr>
</table>

<tr>
<td><div align="left"><font size="11" >1. Images and Pixels</font></div></td>
</tr>

We need to import some modules. We will use the standard `numpy` package to help us with linear algebraic operations on matrices and vectors.

In [ ]:
import numpy as np
import cv2
import sys
import os.path
import math
import pickle

import plotly.express as px
import matplotlib.pyplot as plt

Let's define some helper functions to read, ``iread(relative_path_to_image)``, and display an image ``idisp(image_array)``

In [ ]:
# read an image with colors in RGB order for matplotlib

def iread(filename):
    """
    This function reads an image. Only images in the "images" folder are considered

    :param image: str with name of image to be read. 
    :return: a numpy array of size [image_height, image_width] where each [i,j] corresponds to a pixel in the image.
    """
    return cv2.cvtColor(cv2.imread(os.path.join('Practical06_Support/images', filename)), cv2.COLOR_BGR2GRAY)

# read an image with colors in RGB order for matplotlib
def iread_color(filename):
    """
    This function reads an image. Only images in the "images" folder are considered

    :param image: str with name of image to be read. 
    :return: a numpy array of size [image_height, image_width] where each [i,j] corresponds to a pixel in the image.
    """
    return cv2.cvtColor(cv2.imread(os.path.join('Practical06_Support/images', filename)), cv2.COLOR_BGR2RGB)

def idisp(image, cmap = 'gray', height = None):
    """
    Displaying interactive image.
    """
    labels = dict(x="u (pixels)", y="v (pixels)")
    if height is None: 
        fig = px.imshow(image,color_continuous_scale=cmap, labels = labels)
    else:
        fig = px.imshow(image,color_continuous_scale=cmap, labels = labels, height = height)
    fig.update_layout(coloraxis_showscale=False)
    fig.show()


We will start by loading an image

We will use a convenience function to read the image from a PNG format file.  We can load files of different types (with different extensions), eg. `.jpg`

In [ ]:
image = iread('monalisa.png')
type(image)

and `image` is a numpy array (a python style matrix) with dimensions

In [ ]:
image.shape

which we see has 700 rows and 677 columns.

The data itself is

In [ ]:
image

is simply a big table of 8-bit integers which represent brightness of each pixel as a number between 0 (black) and 1 (white).

We can display it as an image

In [ ]:
idisp(image, height = 600)

**The notebook image view is interactive. If you drift your cursor over the image it displays the pixel coordinate and the grey value of the pixel.**  

In [ ]:
image[0,0]

Common indexing error:

In [ ]:
image[700,0]

Now index is not out of bounds:

In [ ]:
image[677,0]

<table>
  <tr>
    <td><div align="left"><font size="18">2. Basic Image Processing</font></div></td>
  </tr>
</table>

Let's increase the image **brightness**

In [ ]:
added = 120
image2 = image+added 
idisp(image2,height = 600)

Problem: Mona Lisa not looking good:(

Reason: Some pixel values are overflowing!

Solution: cap pixel values to [0,255] interval

But to do this, we need to first cast the image to uint16 ([0, 65535]), then cast it back

In [ ]:
image_16bit = np.array(image.astype(np.uint16))
image_16bit.dtype

### Increasing brightness

In [ ]:
image2 = np.clip(image_16bit+added, 0, 255).astype(np.uint8)
idisp(image2, height = 600)

### Changing contrast

In [ ]:
contrast_factor = 2
image2 = np.clip(image_16bit*contrast_factor, 0, 255).astype(np.uint8)
idisp(image2, height = 600)

### Negative Image

In [ ]:
image2 = 255-image
idisp(image2, height = 600)

### Thresholding

Create a mask based on whether the pixel values is a specific number. 

false (black) if x = Value

true (white) if x != Value

In [ ]:
image_background = iread_color('gs_background.png')
image_greenscreen = iread_color('green_screen.png')
idisp(image_greenscreen, height = 600)

In [ ]:
#what colour is the green pixel?
image_greenscreen[200,200]

In [ ]:
mask = np.alltrue(image_greenscreen == [0,252,0], axis=2)
idisp(mask, height = 600)

In [ ]:
image_masked = image_greenscreen * mask[:, :, None]
#np.broadcast_to(mask, image_greenscreen.shape)
#mask3channel.shape

# (background * mask) + (foreground * inverse mask)
image_background = image_background * mask[:, :, None]
image_foreground = image_greenscreen * np.invert(mask)[:, :, None]

idisp(image_background + image_foreground, height=600)

### Flux Question 1
Which task used diadic image operations?

### Gaussian Blur

In [ ]:
kernel_size = 5 # Window size must be odd!
sigmaX = 5
sigmaY = 5
blur = cv2.GaussianBlur(image,(kernel_size,kernel_size), sigmaX, sigmaY)
idisp(blur, height = 600)

### Flux Question 2
What is the effect of increasing kernel size for Gaussian blur?

<table>
  <tr>
    <td><div align="left"><font size="18">3. Feature Extraction</font></div></td>
  </tr>
</table>

## Corner Detection

In [ ]:
# input image
image_chessboard_color = iread_color('chessboard.jpg')
plt.imshow(image_chessboard_color)

# Convert input img to 32bit, grayscale image
image_chessboard = iread('chessboard.jpg')
image_chessboard = np.float32(image_chessboard)

# Harris corner detector
out = cv2.cornerHarris(image_chessboard,2,3,0.04)

# Threshold for Harris corner detector
corner_threshold = 0.05

circle_radius = 4
circle_color = (255,0,0)
circle_thickness = 0

# Draw circles around the corners for better visibility 
for i in range(out.shape[0]):
    for j in range(out.shape[1]):
        if out[i,j]>corner_threshold*out.max():
            cv2.circle(image_chessboard_color, (j,i), circle_radius, circle_color, circle_thickness)

            
#Display image
plt.figure()
plt.imshow(image_chessboard_color)

## Edge Detection and Line Detection



In [ ]:
# Read input img
image_fed_sq_color = iread_color('empire_state.jpg')
plt.imshow(image_fed_sq_color)

# Canny Edge Detection
#threshold1=first threshold for the hysteresis procedure.
#threshold2=second threshold for the hysteresis procedure.
edges = cv2.Canny(image=image_fed_sq_color, threshold1=100, threshold2=200)
plt.imshow(edges)

# Hough Transform for Line Detection
#rho=Distance resolution of the accumulator in pixels.
#theta=Angle resolution of the accumulator in radians.
#hough_threshold = Accumulator threshold parameter. Only those lines are returned that get enough votes ( >threshold ).
#minLineLength=Minimum line length. Line segments shorter than that are rejected.
#maxLineGap=Maximum allowed gap between points on the same line to link them.
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=100, maxLineGap=10)
for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(image_fed_sq_color,(x1,y1),(x2,y2),(0,255,0),2)
plt.imshow(image_fed_sq_color)


## Feature Descriptors 

In [ ]:
# Get input image
pringles_original_color = iread_color('pringles_can.png')
pringles_original_gray = iread('pringles_can.png')

# Create SIFT Descriptor
sift = cv2.SIFT_create()

# Detect SIFT Keypoints
kp = sift.detect(pringles_original_gray,None)

pringles_original_color=cv2.drawKeypoints(pringles_original_color, kp, pringles_original_color, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(pringles_original_color)

## Feature Matching

In [ ]:
# Get input image
input_img_str = 'pringles_can.png'
pringles_color = iread_color(input_img_str)
pringles_gray = iread(input_img_str)

# Get Test Image
test_img_str = 'pringles_test4.jpg'
pringles_test_color = iread_color(test_img_str)
pringles_test_gray = iread(test_img_str)

# Create SIFT Descriptor
sift = cv2.SIFT_create()

# Detect SIFT Keypoints
kp1, des1 = sift.detectAndCompute(pringles_gray,None)
kp2, des2 = sift.detectAndCompute(pringles_test_gray,None)

# FLANN stands for Fast Library for Approximate Nearest Neighbors
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# store all the good matches as per Lowe's ratio test.
ratio_test_threshold = 0.9
good = []
for m,n in matches:
    if m.distance < ratio_test_threshold*n.distance:
        good.append(m)

#Now we set a condition that atleast 10 matches (defined by MIN_MATCH_COUNT) are to be there to find the object. Otherwise simply show a message saying not enough matches are present.
MIN_MATCH_COUNT = 5

if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = pringles_gray.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    pringles_test_gray = cv2.polylines(pringles_test_gray,[np.int32(dst)],True,255,3, cv2.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None
    
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv2.drawMatches(pringles_color,kp1,pringles_test_color,kp2,good,None,**draw_params)
idisp(img3, height = 600)

### Flux Question 3
Test the system with test images pringles_test1.jpg to pringles_test4.jpg. 

What problems can you observe with keypoint matching based object detection?